In [2]:
## 这个notebook聚焦于 Ablation Study
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


- Ablation Study for Motivation Example

- RAG with HybridRetrieve

In [3]:
def evaluate_predictions(dataset_name, df, dataset):
    acc =0
    error_index = []
    format_error_index = []
    # dataset_name = 'wikitq'
    dataset_df_output = df
    outputs = dataset_df_output.to_dict(orient='index')
    # Iterate through each output and calculate the accuracy
    count = 0
    for i in tqdm(outputs.keys()):
        count += 1
        output = outputs[i]
        pred_answer = None

        # --- Prediction Parsing Logic ---
        if dataset_name == 'wikitq':
            try:
                # Attempt to parse different possible output formats
                raw_output = eval(output['predict'])[0]
                if 'the answer is: ' in raw_output:
                    pred_answer_all = raw_output.split('the answer is: ')[1]
                elif 'Final Answer: ' in raw_output:
                    pred_answer_all = raw_output.split('Final Answer: ')[1]
                else:
                    raise ValueError("Unrecognized format")
                
                # Extract the answer, which is often quoted
                pred_answer = pred_answer_all.split('"')[1:2]

            except Exception as e:
                # If parsing fails, log it as a format error
                # print(f'Format error in index {i}: {e}')
                format_error_index.append(i)
                pred_answer = [''] # Assign an empty prediction
                pred_answer_all = ''
                # print(pred_answer)

        elif dataset_name == 'tab_fact':
            # For TabFact, check for 'true' or 'false' (case-insensitive)
            # if isinstance(output['predict'], str) and 'true' in output['predict'].lower():
            try:
                pred_answer_str = eval(output['predict'])[0].lower().split('statement is: ')[1].replace(" ", "")
            except:
                pred_answer_str = eval(output['predict'])[0]
            if 'false' in pred_answer_str:
                pred_answer = [0]
            else:
                pred_answer = [1]

            # gold_answer = result_df[str(i)]['ori_data_item']['answer_text']
        gold_answer = dataset[i]['answer_text']
        # Score is either 1 or 0
        # print(pred_answer)
        score = Evaluator().evaluate(
        pred_answer,
        gold_answer,
        dataset=dataset_name,
        question=output['question']
        )
        if dataset_name=='wikitq':
            if score == False and isinstance(pred_answer_all, str):
                score = Evaluator().evaluate(
                    pred_answer_all.split(','),
                    gold_answer,
                    dataset=dataset_name,
                    question=output.get('question', '') # Safely get question if available
                )
        acc += score
        if score != 1:
            error_index.append(i)
            # if score!=1:
            #     print(f'The prediction is {pred_answer}, while the ground truth is {gold_answer} for sample {i}.')
        # except:
        #     print(i)
        #     pass
    final_accuracy = 100 * acc / count if count > 0 else 0
    print(f"Correct Samples: {acc}; Total Samples: {count}")
    print(f"Accuracy: {100*acc/count:.2f}")
    return final_accuracy, error_index, format_error_index

In [58]:
### 读取必要文件 （step 2）
dataset_name = 'wikitq'
split = 'validation'
dataset = load_data_split(dataset_name,split)
# wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()
wikitq_df_processed = np.load('/data/workspace/yanmy/HybridRAG/H-STAR/datasets/wikitq_df_validation.npy', allow_pickle=True).item()
assert len(dataset) == len(wikitq_df_processed)
wikitq_df = pd.DataFrame(dataset)
long_index = np.load('../datasets/pipeline/valid_data_selection.npy',allow_pickle=True)

In [4]:
dataset_name = 'tab_fact'
split = 'test_small'
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()
# wikitq_df_processed = np.load('/data/workspace/yanmy/HybridRAG/H-STAR/datasets/wikitq_df_validation.npy', allow_pickle=True).item()
assert len(dataset) == len(wikitq_df_processed)
wikitq_df = pd.DataFrame(dataset)
# long_index = np.load('../datasets/pipeline/valid_data_selection.npy',allow_pickle=True)

In [7]:
prompt_list = []
for index in range(len(dataset)):
    index = int(index)
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/rewrite_query.txt',processed=True)
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list
wikitq_df_rewrite = wikitq_df
wikitq_df_rewrite['instruction'] = prompt_list

In [10]:
wikitq_df_rewrite.to_csv('../datasets/ablation/tab_fact_test_small_rewrite.csv')

In [14]:
rewrite_result_df = pd.read_csv('../datasets/ablation/tab_fact_test_small_rewrite_output_4B.csv',index_col=0)
rewrite_result = {}
# rewrite_result = [eval(item)[0] for item in rewrite_result['predict'].tolist()]
for index in range(len(dataset)):
    rewrite_result[index] = eval(rewrite_result_df.iloc[index,-1])[0]

In [15]:
np.save('datasets/schedule_test/tab_fact/rewrite_query.npy',rewrite_result)

In [67]:
wikitq_df_rewrite.to_csv('../datasets/ablation/wikitq_validation_rewrite.csv')

In [68]:
rewrite_result = pd.read_csv('../datasets/ablation/wikitq_validation_rewrite_output_4B.csv',index_col=0)

In [77]:
rewrite_query_list = {}
for index in long_index:
    index = int(index)
    rewritten_query = eval(rewrite_result.loc[index,'predict'])[0]
    # dataset[index]['question'] = rewritten_query
    # rewrite_query_list.append(rewritten_query)
    rewrite_query_list[index] = f"[ORIGINAL]: {dataset[index]['question']}  \n" + rewritten_query
    # wikitq_df.at[index,'rewritten_query'] = rewritten_query

In [81]:
RAG_Hybrid = np.load('../datasets/ablation/wikitq_valid_RAG_Hybrid.npy',allow_pickle=True).item()

In [133]:
RAG_Semantic = np.load('../datasets/ablation/wikitq_valid_RAG_Hybrid_Semantic.npy',allow_pickle=True).item()

In [47]:
np.save('../datasets/ablation/wikitq_validation_rewrite_query_list_4B.npy',rewrite_query_list)

In [ ]:
python Hybrid_Retrieve_Update_dict.py \
    --model_path /data/workspace/yanmy/models/bge-m3 \
    --dataset_name wikitq \
    --split validation \
    --index_path ../datasets/pipeline/valid_data_selection.npy \
    --output_path ../datasets/ablation/wikitq_valid_RAG_Hybrid.npy \
    --max_rows 50 \
    --max_cols 10 \
    --processed_df_path ../datasets/wikitq_df_validation.npy \
    --rewrite_query_path ../datasets/ablation/wikitq_validation_rewrite_query_list_4B.npy

In [ ]:
python Hybrid_Retrieve_Update_dict.py \
    --model_path /data/workspace/yanmy/models/bge-m3 \
    --dataset_name tab_fact \
    --split test_small \
    --output_path ../datasets/ablation/tab_fact_test_small_RAG_Hybrid.npy \
    --max_rows 50 \
    --max_cols 10 \
    --processed_df_path datasets/schedule_test/tab_fact/wikitq_df_processed.npy \
    --rewrite_query_path datasets/schedule_test/tab_fact/rewrite_query.npy

In [ ]:
python Hybrid_Retrieve_Semantic.py \
    --model_path /data/workspace/yanmy/models/bge-m3 \
    --dataset_name wikitq \
    --split validation \
    --index_path ../datasets/pipeline/valid_data_selection.npy \
    --output_path ../datasets/ablation/wikitq_valid_RAG_Hybrid_Semantic.npy \
    --max_rows 50 \
    --max_cols 10 \
    --processed_df_path ../datasets/wikitq_df_validation.npy 

In [128]:
prompt_list = []
for index in long_index:
    index = int(index)
    # rewritten_query = rewrite_query_list[index]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    # insert_df = RAG_Hybrid[index]
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/generic_reasoning/chain_of_thought_wtq.txt',processed=True)
    # prompt = prompt.replace(dataset[index]['question'], rewritten_query.split('SPECIFIC')[0][:-1])
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt)
wikitq_df_qa = wikitq_df.iloc[long_index]
wikitq_df_qa['instruction'] = prompt_list

/tmp/ipykernel_1640787/1094960381.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikitq_df_qa['instruction'] = prompt_list


In [134]:
prompt_list = []
for index in long_index:
    index = int(index)
    rewritten_query = rewrite_query_list[index]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    insert_df = RAG_Hybrid[index]
    prompt = build_wikitq_prompt_from_df(dataset,RAG_Semantic[index],index,template_path='../prompts/generic_reasoning/chain_of_thought_wtq.txt',processed=True)
    # prompt = prompt.replace(dataset[index]['question'], rewritten_query.split('SPECIFIC')[0][:-1])
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt)
wikitq_df_rewrite_qa_semantic = wikitq_df.iloc[long_index]
wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list

/tmp/ipykernel_1640787/2915256273.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list


In [135]:
wikitq_df_rewrite_qa_semantic.to_csv('../datasets/ablation/wikitq_validation_qa_semantic.csv')

In [114]:
prompt_list = []
for index in long_index:
    index = int(index)
    rewritten_query = rewrite_query_list[index]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    insert_df = RAG_Hybrid[index]
    prompt = build_wikitq_prompt_from_df(dataset,RAG_Hybrid[index],index,template_path='../prompts/text_reason_wtq.txt',processed=True)
    # prompt = prompt.replace(dataset[index]['question'], rewritten_query.split('SPECIFIC')[0][:-1])
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt + sql_execute_result.iloc[index,-2].split('<output>')[-1])
wikitq_df_rewrite_qa = wikitq_df.iloc[long_index]
wikitq_df_rewrite_qa['instruction'] = prompt_list

/tmp/ipykernel_1640787/1816279342.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikitq_df_rewrite_qa['instruction'] = prompt_list


In [129]:
wikitq_df_qa.to_csv('../datasets/ablation/wikitq_validation_qa.csv')

In [119]:
wikitq_df_rewrite_qa.to_csv('../datasets/ablation/wikitq_validation_RAG_Hybrid_noSpecific_qa.csv')

In [111]:
sql_execute_result = pd.read_csv('../datasets/pipeline/wikitq_valid_sql_exec_qa_output_4B.csv')

In [186]:
# wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_validation_RAG_Hybrid_noRewrite_qa_output_4B.csv',index_col=0)
# wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_validation_RAG_Hybrid_noSpecific_qa_output_4B.csv',index_col=0)
# wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_validation_qa_output_4B.csv',index_col=0)
# wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_validation_qa_semantic_output_4B.csv',index_col=0)
wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_valid_training_free_qa_output_4B.csv',index_col=0)
final_accuracy, error_index, format_error_index = evaluate_predictions('wikitq', wikitq_df_rewrite_qa_output, dataset)

  0%|          | 0/476 [00:00<?, ?it/s]

100%|██████████| 476/476 [00:14<00:00, 31.82it/s]

Correct Samples: 237; Total Samples: 476
Accuracy: 49.79


In [151]:
filtered_table_valid_row = np.load('../datasets/pipeline/filtered_tables_row.npy',allow_pickle=True).item()
filtered_table_valid_col = np.load('../datasets/pipeline/filtered_tables_col.npy',allow_pickle=True).item()

In [159]:
row_success_count = 0
col_success_count = 0
sql_success_count = 0
# len(filtered_table_valid_row),len(filtered_table_valid_col)
for index in long_index:
    index = int(index)
    if len(filtered_table_valid_row[index]) < len(wikitq_df_processed[index]):
        row_success_count += 1
    if filtered_table_valid_col[index].shape[1] < wikitq_df_processed[index].shape[1]:
        col_success_count += 1
    sql_input = sql_execute_result.iloc[index,-2].split('<output>')[-1] ## template+table+qustion 的末尾
    if sql_input.__contains__('additional evidence'):
        sql_success_count += 1
print(f'{1 - row_success_count/len(long_index)} row parse fail, {1 - col_success_count/len(long_index)} col parse fail, {1 - sql_success_count/len(long_index)} sql contain additional evidence')
    

0.2647058823529411 row parse fail, 0.025210084033613467 col parse fail, 0.21638655462184875 sql contain additional evidence


In [ ]:
### SQL success parse

In [28]:
success_sql = 0
all_sql  = 0
processed_table = np.load('datasets/schedule_test/wikitq/processed_table.npy',allow_pickle=True).item()
filtered_table_test_row = processed_table['Select_Row']
filtered_table_test_col = processed_table['Select_Column']
wikitq_df_processed = np.load('datasets/schedule_test/wikitq/wikitq_df_processed.npy',allow_pickle=True).item()
sql_execute_result = processed_table['Execute_SQL']
for key in filtered_table_test_row.keys():
    all_sql += 1
    if len(filtered_table_test_row[key]) < len(wikitq_df_processed[key]):
        success_sql += 1
for key in filtered_table_test_col.keys():
    all_sql += 1
    if filtered_table_test_col[key].shape[1] < wikitq_df_processed[key].shape[1]:
        success_sql += 1
for key in sql_execute_result.keys():
    all_sql += 1
    if len(sql_execute_result[key])>0:
        success_sql += 1
# sql_execute_result = np.load('')
# assert len(filtered_table_test_row) == len(filtered_table_test_col)

In [29]:
success_sql / all_sql

0.7777998017839445

In [160]:
wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/pipeline/wikitq_valid_row_sql_qa_output_4B.csv',index_col=0).iloc[long_index]
wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/pipeline/wikitq_valid_col_sql_qa_output_4B.csv',index_col=0).iloc[long_index]
final_accuracy, error_index, format_error_index = evaluate_predictions('wikitq', wikitq_df_rewrite_qa_output, dataset)

100%|██████████| 476/476 [00:09<00:00, 50.23it/s]

Correct Samples: 282; Total Samples: 476
Accuracy: 59.24


In [ ]:
df = pd.read_csv('../datasets/pipeline/')

### Training-Free Router

In [161]:
df = pd.read_csv('/data/workspace/yanmy/HybridRAG/H-STAR/router/test_data_retrieved.csv',index_col=0)

In [164]:
## generate training-free router result
prompt_list = []
for index in long_index:
    index = int(index)
    # rewritten_query = rewrite_query_list[index]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    # insert_df = RAG_Hybrid[index]
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/training_free_router.txt',processed=True)
    # prompt = prompt.replace(dataset[index]['question'], rewritten_query.split('SPECIFIC')[0][:-1])
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt)
wikitq_df_rewrite_qa_semantic = wikitq_df.iloc[long_index]
wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list

/tmp/ipykernel_1640787/1807860779.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list


In [168]:
wikitq_df_rewrite_qa_semantic.to_csv('../datasets/ablation/wikitq_valid_training_free_router.csv')

In [169]:
training_tree_result = pd.read_csv('../datasets/ablation/wikitq_valid_training_free_router_output_4B.csv',index_col=0)

In [223]:
# eval(training_tree_result.iloc[221,-1])[0]
def parse_valid_route(input):
    parse = eval(input)[0]
    return eval(parse)
prompt_list = []
count = 0
count_prun = 0
count_original = 0
count_seq_len = 0
for index in long_index:
    index = int(index)
    route = parse_valid_route(training_tree_result.loc[index,'predict'])
    # count_seq_len += len(route)

    # if 'Select_Column' in route and 'Select_Row' in route:
    #     df_row = filtered_table_valid_row[index]
    #     df_col = filtered_table_valid_col[index]
    #     df_final = find_intersection_and_add_row_id(df_row,df_col)
    #     count += 4
    # elif 'Select_Column' in route:
    #     df_final = filtered_table_valid_col[index]
    #     count += 2
    # elif 'Select_Row' in route:
    #     df_final = filtered_table_valid_row[index]
    #     count += 2
    # else:
    #     df_final = wikitq_df_processed[index]
    # if 'RAG' in route:
    #     df_final = find_intersection_and_add_row_id(df_final,RAG_Hybrid[index])
    if 'Select_Row' in route:
        df_final = filtered_table_valid_row[index]
        count += 2
        count_seq_len += 1
    else:
        df_final = wikitq_df_processed[index]
    prompt = build_wikitq_prompt_from_df(dataset,df_final,index,template_path='../prompts/text_reason_wtq.txt',processed=True)
    count_prun += df_final.shape[0] * df_final.shape[1]
    count_original += wikitq_df_processed[index].shape[0] * wikitq_df_processed[index].shape[1]
    if 'Execute_SQL' in route:
        sql_input = sql_execute_result.iloc[index,-2].split('<output>')[-1]
        prompt = prompt + sql_input
        count += 3
        count_seq_len += 1
    prompt_list.append(prompt)
wikitq_df_training_free_qa = wikitq_df.iloc[long_index]
wikitq_df_training_free_qa['instruction'] = prompt_list

/tmp/ipykernel_1640787/4259182048.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikitq_df_training_free_qa['instruction'] = prompt_list


In [224]:
count_seq_len

602

In [ ]:
0.7330156694619411 router 994 Full Route
0.5642 router + check 602

In [212]:
count / len(long_index) + 2

6.260504201680672

In [211]:
wikitq_df_training_free_qa.to_csv('../datasets/ablation/wikitq_valid_training_free_qa_route.csv')

In [213]:
# wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_valid_training_free_qa_output_4B.csv',index_col=0)
wikitq_df_rewrite_qa_output = pd.read_csv('../datasets/ablation/wikitq_valid_training_free_qa_route_output_4B.csv',index_col=0)
final_accuracy, error_index, format_error_index = evaluate_predictions('wikitq', wikitq_df_rewrite_qa_output, dataset)

  0%|          | 0/476 [00:00<?, ?it/s]

100%|██████████| 476/476 [00:09<00:00, 52.24it/s]

Correct Samples: 304; Total Samples: 476
Accuracy: 63.87


In [216]:
row_num = []
col_num = []
for index in long_index:
    index = int(index)
    row_num.append(wikitq_df_processed[index].shape[0])
    col_num.append(wikitq_df_processed[index].shape[1])
print(f'average row number is {np.mean(row_num)}, average col number is {np.mean(col_num)},maximum row is {np.max(row_num)}, maximum col is {np.max(col_num)}')

average row number is 78.73529411764706, average col number is 7.571428571428571,maximum row is 753, maximum col is 21
